In [1]:
import pandas as pd
import os
import librosa
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pickle
import h5py
import joblib

with open('/kaggle/input/femalepickle/female_mfcc.pickle', 'rb') as file:
     X = pickle.load(file)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
print(X.shape)

(151093, 16, 5)


In [3]:
X.shape

(151093, 16, 5)

In [4]:
import pandas as pd
import os
import librosa
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pickle
import h5py
import joblib

df = pd.read_csv('/kaggle/input/trainnnnnn/train_test.csv')

def map_age_to_range(age):
    if age in ['teens', 'twenties']:
        return 'teens-twenties'
    elif age in ['thirties', 'forties', 'fifties']:
        return 'thirties-fifties'
    else:
        return 'sixties-eighties'

df['age range'] = df['age'].apply(map_age_to_range)
female_df = df[df['gender'] == 'female']

error_list = []

# Open the text file and read its contents
with open('/kaggle/input/femalepickle/error files.txt', 'r') as file:
    error_list = file.read().splitlines()

In [5]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

female_df = female_df[~female_df['path'].isin(error_list)]
        
y = female_df['age range'].values

# Fit the encoder on your target labels and transform them to integers
y = label_encoder.fit_transform(y)

with open('label_encoder.pkl', 'wb') as label_encoder_file:
    pickle.dump(label_encoder, label_encoder_file)

In [6]:
# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Define a feedforward neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(16, 5, 1)),  # Adjusted input shape
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # Adjust the output layer according to your problem
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=32)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

# Save the model in different formats
model.save('female_age_cnn_model.h5')
with open('female_age_cnn_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)
joblib.dump(model, 'female_age_cnn_model.joblib')

Epoch 1/200
2833/2833 [==============================] - 13s 3ms/step - loss: 2.3227 - accuracy: 0.3920 - val_loss: 1.1627 - val_accuracy: 0.4262
Epoch 2/200
2833/2833 [==============================] - 9s 3ms/step - loss: 1.2630 - accuracy: 0.4169 - val_loss: 1.2267 - val_accuracy: 0.3867
Epoch 3/200
2833/2833 [==============================] - 9s 3ms/step - loss: 1.1002 - accuracy: 0.4431 - val_loss: 1.0664 - val_accuracy: 0.4147
Epoch 4/200
2833/2833 [==============================] - 9s 3ms/step - loss: 1.0409 - accuracy: 0.4624 - val_loss: 1.0353 - val_accuracy: 0.4746
Epoch 5/200
2833/2833 [==============================] - 9s 3ms/step - loss: 1.0171 - accuracy: 0.4771 - val_loss: 1.0114 - val_accuracy: 0.4772
Epoch 6/200
2833/2833 [==============================] - 9s 3ms/step - loss: 1.0034 - accuracy: 0.4863 - val_loss: 0.9924 - val_accuracy: 0.4917
Epoch 7/200
2833/2833 [==============================] - 9s 3ms/step - loss: 0.9946 - accuracy: 0.4943 - val_loss: 1.0013 - val_a

['female_age_cnn_model.joblib']